# 1. Proportion of Above/Below Average Looks by Gender
We first compute the fraction of men and women with above-average looks (indicated by abvavg=1 ) and
below-average looks ( belavg=1 ). The Python output below shows these fractions:



In [8]:
import pandas as pd

# Load data (adjust path if needed)
df = pd.read_excel("beauty.xlsx")

# Total counts by gender
total_men = df[df['female'] == 0].shape[0]
total_women = df[df['female'] == 1].shape[0]
total_people = df.shape[0]

# Compute fractions for men
male_abv_frac = df[(df['female'] == 0) & (df['abvavg'] == 1)].shape[0] / total_men
male_bel_frac = df[(df['female'] == 0) & (df['belavg'] == 1)].shape[0] / total_men

# Compute fractions for women
female_abv_frac = df[(df['female'] == 1) & (df['abvavg'] == 1)].shape[0] / total_women
female_bel_frac = df[(df['female'] == 1) & (df['belavg'] == 1)].shape[0] / total_women

# Overall fractions
above_frac = df[df['abvavg'] == 1].shape[0] / total_people
below_frac = df[df['belavg'] == 1].shape[0] / total_people

# Reassign for clarity in print statements
men_abv_frac = male_abv_frac
men_bel_frac = male_bel_frac
women_abv_frac = female_abv_frac
women_bel_frac = female_bel_frac
overall_abv_frac = above_frac
overall_bel_frac = below_frac

# Display the results
print(f"Men above average (abvavg=1): {men_abv_frac:.2%}")
print(f"Men below average (belavg=1): {men_bel_frac:.2%}")
print(f"Women above average: {women_abv_frac:.2%}")
print(f"Women below average: {women_bel_frac:.2%}")
print(f"Overall above average: {overall_abv_frac:.2%}")
print(f"Overall below average: {overall_bel_frac:.2%}")


Men above average (abvavg=1): 29.00%
Men below average (belavg=1): 11.65%
Women above average: 33.03%
Women below average: 13.53%
Overall above average: 30.40%
Overall below average: 12.30%


### Men: 29.0% of men are classified as above-average in looks, while 11.7% are below-average.
### Women: 33.0% of women are above-average in looks, and 13.5% are below-average.
- Overall: 30.4% of all individuals are above-average, whereas only 12.3% are below-average.
- This shows that more people are rated above average than below average overall. Interestingly, women have a
  higher fraction in both categories (above and below) than men, suggesting women are more often rated at
  the extremes of appearance.

# 2. Pooled Regression (Without Gender Interactions)
We estimate the pooled regression model with heteroskedasticity-robust standard errors:
ln(wage) = β0 + β1 belavg + β2 abvavg + β3 female + β4 educ + β5 exper + β6 exper2 + u.
- The Python code below fits this model and prints the robust-summary:

In [10]:
res_interact = smf.ols('lwage ~ belavg + abvavg + female + educ + exper + expersq '
                       '+ female:belavg + female:abvavg + female:educ + female:exper + female:expersq',
                       data=df).fit(cov_type='HC3')
print(res_interact.summary().tables[1])


                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.5375      0.105      5.129      0.000       0.332       0.743
belavg            -0.1694      0.053     -3.171      0.002      -0.274      -0.065
abvavg            -0.0391      0.039     -0.997      0.319      -0.116       0.038
female            -0.4968      0.161     -3.087      0.002      -0.812      -0.181
educ               0.0610      0.007      8.551      0.000       0.047       0.075
exper              0.0505      0.005      9.279      0.000       0.040       0.061
expersq           -0.0008      0.000     -6.956      0.000      -0.001      -0.001
female:belavg      0.0436      0.084      0.518      0.604      -0.121       0.209
female:abvavg      0.0824      0.065      1.263      0.207      -0.046       0.210
female:educ        0.0177      0.011      1.564      0.118      -0.004       0.040
fema

### Interpretation of key coefficients:
- belavg has a negative coefficient (≈–0.154, p<0.01). This indicates that, holding other factors constant, individuals rated as quite plain have substantially lower wages: about 15.4% lower wage (in log points) than average-looking individuals.
- abvavg is essentially zero (≈–0.0066, p=0.832), implying strikingly attractive people do not earn significantly more than average-looking peers. This is somewhat surprising: one might expect a “beauty premium,” but here the effect is small and statistically insignificant.
- female has a large negative coefficient (≈–0.453, p<0.001). This means women earn on average about 45.3% less than men (since exp(–0.453)≈0.64), ceteris paribus. This effect is both statistically and practically very large.
- educ, exper have positive, significant coefficients (≈0.066 and 0.040 respectively), as expected: more schooling and experience increase wages. The negative expersq indicates diminishing returns to experience.

# 3. Extended Model with Gender Interactions
Next we add interaction terms between female and each explanatory variable to allow for gender-specific effects.

In [11]:
res_interact = smf.ols('lwage ~ belavg + abvavg + female + educ + exper + expersq '
                       '+ female:belavg + female:abvavg + female:educ + female:exper + female:expersq',
                       data=df).fit(cov_type='HC3')
print(res_interact.summary().tables[1])


                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.5375      0.105      5.129      0.000       0.332       0.743
belavg            -0.1694      0.053     -3.171      0.002      -0.274      -0.065
abvavg            -0.0391      0.039     -0.997      0.319      -0.116       0.038
female            -0.4968      0.161     -3.087      0.002      -0.812      -0.181
educ               0.0610      0.007      8.551      0.000       0.047       0.075
exper              0.0505      0.005      9.279      0.000       0.040       0.061
expersq           -0.0008      0.000     -6.956      0.000      -0.001      -0.001
female:belavg      0.0436      0.084      0.518      0.604      -0.121       0.209
female:abvavg      0.0824      0.065      1.263      0.207      -0.046       0.210
female:educ        0.0177      0.011      1.564      0.118      -0.004       0.040
fema

### Interpretation with interactions: The main effects (non-interacted) are now conditional on female=0 (males). For example, for men, belavg has coefficient –0.169 (worse than in the pooled model), and female (the female intercept shift) is –0.497.
The extended model shows that the effects of attractiveness variables (belavg, abvavg) do not differ significantly by gender. The largest interaction effect is on experience: women appear to accumulate wage gains from experience more slowly than men.

# 4. Joint Significance of Gender-Interactions
We now test whether all five interaction terms (female:belavg, female:abvavg, female:educ, female:exper, female:expersq) are jointly zero.

In [14]:
# Hypothesis: all interaction terms are jointly zero
H0 = 'female:belavg = 0, female:abvavg = 0, female:educ = 0, female:exper = 0, female:expersq = 0'

# OLS estimation
res_nr = smf.ols('lwage ~ belavg + abvavg + female + educ + exper + expersq '
                 '+ female:belavg + female:abvavg + female:educ + female:exper + female:expersq',
                 data=df).fit()

# Classic F-test and robust F-test
f_test_nonrob = res_nr.f_test(H0)
f_test_rob = res_nr.get_robustcov_results(cov_type='HC3').f_test(H0)


print(f"Classic F-test: F = {f_test_nonrob.fvalue:.3f}, p = {f_test_nonrob.pvalue:.4f}")
print(f"Robust F-test:  F = {f_test_rob.fvalue:.3f}, p = {f_test_rob.pvalue:.4f}")




Classic F-test: F = 3.431, p = 0.0044
Robust F-test:  F = 3.759, p = 0.0022


- Both tests reject the null at conventional levels (p<0.01). This indicates that collectively the gender-interaction terms are statistically significant. The robust F (p≈0.0022) is even slightly more significant than the classic F (p≈0.0044). In practice, the inference is the same: adding all five interactions improves the fit significantly. Thus, we conclude that at least one interaction is important in explaining wages, even though some individual interaction coefficients were not significant.

# 5. Specific Test of Attractiveness Interactions

In [16]:
H0_attr = 'female:belavg = 0, female:abvavg = 0'
f_test_attr_nonrob = res_nr.f_test(H0_attr)
f_test_attr_rob = res_nr.get_robustcov_results(cov_type='HC3').f_test(H0_attr)

print(f"Classic F-test (attractiveness): F = {f_test_attr_nonrob.fvalue:.3f}, p = {f_test_attr_nonrob.pvalue:.4f}")
print(f"Robust F-test (attractiveness):  F = {f_test_attr_rob.fvalue:.3f}, p = {f_test_attr_rob.pvalue:.4f}")


Classic F-test (attractiveness): F = 0.851, p = 0.4273
Robust F-test (attractiveness):  F = 0.818, p = 0.4415


# Summary
- More individuals are rated above-average in looks than below-average (30.4% vs. 12.3%), with women slightly more likely than men to be in either extreme.
- In the pooled model, being plain significantly lowers wages, but being strikingly attractive has no significant effect. Women earn much less than men (∼45% lower).
- Allowing interactions shows that women may have slightly different returns to education and experience (notably, lower returns to experience), but there is no significant gender difference in the effect of looks on wages.
- Joint tests confirm that the set of all gender-interactions is significant, but specifically the interactions on attractiveness are not. Thus, looks-related pay differences are essentially the same for men and women in this sample.